In [90]:
#r "nuget: Plotly.NET.Interactive"
#r "nuget: Plotly.NET.ImageExport"
#load "./Camara.fsx"

open System
open Camara
open Html

let baseUrl = "https://camaradosdeputadosgql-qboe6n5gda-uc.a.run.app/graphql"
//let baseUrl = "https://localhost:7043/graphql"
let client = Camara.Client(baseUrl)

Installed Packages Plotly.NET.ImageExport, 5.0.1 Plotly.NET.Interactive, 4.2.1

In [92]:
// Lista todos os deputados da atual legislatura e suas informações básicas em uma tabela
let view (deputies:Model.Deputy[]) = 
    table [] [
        tr [] [
            th [] [ "Id" ]
            th [] [ "Nome" ]
            th [] [ "Partido" ]
            th [] [ "Estado" ]
            th [] [ "" ]
        ]
        for deputy in deputies do
            tr [] [
                td [] [ deputy.id ]
                td [] [ deputy.name ]
                td [] [ deputy.party ]
                td [] [ deputy.state ]
                td [] [ img [ _src deputy.picture; _width "32px" ] [] ]
            ]
    ]

In [93]:
let YEAR = 2023
let MONTH = Nullable<int>()
let deputies = client.GetDeputyExpenses(204507, YEAR, MONTH).deputies
display ( view deputies )

Id,Nome,Partido,Estado,
204507,Carla Zambelli,PL,SP,


In [94]:
deputies.[0].expenses |> Array.length

472

In [95]:
// Funções para plotar gráficos	

open Plotly.NET
open Plotly.NET.LayoutObjects
open Plotly.NET.ImageExport

let logValueAxis = LinearAxis.init(
    Title=Title.init(Text = "Valor (R$)")
    )

let expenseTypeOverTime (deputy:Model.Deputy) =
    if Array.isEmpty deputy.expenses then
        Chart.StackedArea([||], [||])
    else
    deputy.expenses
    |> Seq.groupBy (fun e -> e.expenseType)
    |> Seq.map(fun (expenseType, expenses) -> 
        let dates, values =
            expenses
            |> Seq.filter(fun it -> it.documentDate.HasValue)
            |> Seq.sortBy(fun it -> it.documentDate.Value)
            |> Seq.map(fun it -> it.documentDate.Value.Date, it.netValue)
            |> Seq.toArray
            |> Array.unzip
        Chart.StackedArea(dates, values, Name=expenseType)
        |> Chart.withLineStyle(Shape = StyleParam.Shape.Hvh)
    )
    |> Chart.combine
    |> Chart.withYAxis logValueAxis
    |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
    |> Chart.withLayoutStyle(Width = 1000, Height = 600)
    |> Chart.withTitle($"{deputy.name} - {deputy.party} - {deputy.state} - {MONTH}/{YEAR}")

let expenseTypePieChart (deputy:Model.Deputy) =
    
    deputy.expenses
    |> Seq.groupBy (fun e -> e.expenseType)
    |> Seq.map(fun (expenseType, expenses) -> 
        let totalValue = expenses |> Seq.sumBy(fun it -> it.netValue)
        expenseType, totalValue
    )
    |> Seq.toArray
    |> Array.unzip
    |> fun (labels, values) -> 
        let total = values |> Array.sum
        Chart.Pie(values=values, Labels=labels)
        |> Chart.withTitle($"{deputy.name} - {deputy.party} - {deputy.state} - R$ {total} - {MONTH}/{YEAR}")
        |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)

In [96]:
deputies.[0]
|> expenseTypePieChart
|> display

<!-- Plotly chart will be drawn inside this DIV -->

In [97]:
deputies.[0]
|> expenseTypeOverTime
|> display

<!-- Plotly chart will be drawn inside this DIV -->

In [20]:
let legislature = client.GetLegislature(DateTime.Now).legislatures.[0]
display (view (legislature.deputies))

Id,Nome,Partido,Estado,
220593,Abilio Brunini,PL,MT,
204379,Acácio Favacho,MDB,AP,
220714,Adail Filho,REPUBLICANOS,AM,
221328,Adilson Barroso,PL,SP,
204560,Adolfo Viana,PSDB,BA,
204528,Adriana Ventura,NOVO,SP,
121948,Adriano do Baldy,PP,GO,
74646,Aécio Neves,PSDB,MG,
160508,Afonso Florence,PT,BA,
136811,Afonso Hamm,PP,RS,
